In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import layers
import os

from process_ntuples import *

In [12]:
dir_path = "/global/cfs/cdirs/atlas/gottino/Charm/Analysis/run_batch_gup/ntuples"

files = os.listdir(dir_path)
TTree = process_ntuple(dir_path, files[0])
files

['CharmAnalysis.mc.ForcedDecay.Dplus.tree.root',
 'CharmAnalysis.mc.ForcedDecay.LabmdaC.tree.root']

$D^{+}$ : 411

$\Lambda_C^{+}$ : 4122

In [13]:
DMesons_df = show(0, TTree, ["DMesons_pt", "DMesons_m", "DMesons_eta", "DMesons_fitOutput__Lxy", "DMesons_costhetastar", "DMesons_pdgId"], 100)
DMesons_df = remove_empty(DMesons_df)

DMesons_df["DMesons_pt"] = DMesons_df["DMesons_pt"].apply(divide_by_1000)
for col in DMesons_df.columns:
    DMesons_df[col] = DMesons_df[col].apply(list)
DMesons_df

,DMesons_pt,DMesons_m,DMesons_eta,DMesons_fitOutput__Lxy,DMesons_costhetastar,DMesons_pdgId
0,"[5.73574853515625, 5.73574853515625]","[1860.0809326171875, 2207.808349609375]","[0.9682132005691528, 0.9682132005691528]","[3.940532684326172, 3.940532684326172]","[-0.7892730832099915, -0.41035810112953186]","[411, 4122]"
1,[10.047125],[1862.4176025390625],[0.08182407170534134],[2.3395302295684814],[0.25487038493156433],[411]
2,"[18.9937421875, 18.9937421875]","[1853.90625, 2398.886962890625]","[-1.6619627475738525, -1.6619627475738525]","[1.9184143543243408, 1.9184143543243408]","[-0.36730244755744934, -0.0650731548666954]","[411, 4122]"
3,"[10.118494140625, 10.118494140625]","[1874.203125, 2468.1806640625]","[0.041659336537122726, 0.041659336537122726]","[2.8901257514953613, 2.8901257514953613]","[0.06833142042160034, 0.4276972711086273]","[411, 4122]"
4,"[12.579470703125, 12.579470703125]","[1884.987060546875, 2220.384521484375]","[-1.1025604009628296, -1.1025604009628296]","[0.9249186515808105, 0.9249186515808105]","[-0.5603189468383789, -0.41332852840423584]","[411, 4122]"
...,...,...,...,...,...,...
60,"[12.4448828125, 12.4448828125]","[1915.7474365234375, 2336.346435546875]","[-1.1050832271575928, -1.1050832271575928]","[0.9720778465270996, 0.9720778465270996]","[0.6463268399238586, 0.8104363679885864]","[411, 4122]"
61,[20.794609375],[2308.2685546875],[-0.20504680275917053],[0.21821393072605133],[-0.09182504564523697],[4122]
62,"[30.92643359375, 30.92643359375]","[1892.5665283203125, 2442.986572265625]","[1.0249760150909424, 1.0249760150909424]","[14.670319557189941, 14.670319557189941]","[0.35485926270484924, 0.5878831148147583]","[411, 4122]"
63,[6.7828095703125],[2423.11767578125],[0.6877735257148743],[0.3555510938167572],[0.003152142744511366],[4122]


In [9]:
#only get Lambda
# count = 0
# for index, row in DMesons_df.iterrows():
#     if count == 50:
#         break
#     count+=1
    
#     pts = row['DMesons_daughterInfo__pt']
#     etas = row['DMesons_daughterInfo__eta']
#     phis = row['DMesons_daughterInfo__phi']
#     pdgids = row["DMesons_daughterInfo__pdgId"]
    

Apply Cuts Fiducial Region cuts ($𝐷^{+}$ $𝑝_T$ > 8 GeV, |𝜂| < 2.2)

In [14]:
cut_indicies = []
for index, row in DMesons_df.iterrows():
    pts = row["DMesons_pt"]
    ms = row["DMesons_m"]
    etas = row["DMesons_eta"]    
    count = 0
    for pt, m, eta in zip(pts, ms, etas):
        if count == 1:
            count = 0
            continue
        if (pt < 8):
            cut_indicies.append(index)
        elif (m < 100):
            cut_indicies.append(index)
        elif (eta > 2.2):
            cut_indicies.append(index)
        count+=1

cut_indicies

DMesons_df = DMesons_df.drop(index = cut_indicies, axis = 0)
DMesons_df = DMesons_df.reset_index(drop=True)

DMesons_df

,DMesons_pt,DMesons_m,DMesons_eta,DMesons_fitOutput__Lxy,DMesons_costhetastar,DMesons_pdgId
0,[10.047125],[1862.4176025390625],[0.08182407170534134],[2.3395302295684814],[0.25487038493156433],[411]
1,"[18.9937421875, 18.9937421875]","[1853.90625, 2398.886962890625]","[-1.6619627475738525, -1.6619627475738525]","[1.9184143543243408, 1.9184143543243408]","[-0.36730244755744934, -0.0650731548666954]","[411, 4122]"
2,"[10.118494140625, 10.118494140625]","[1874.203125, 2468.1806640625]","[0.041659336537122726, 0.041659336537122726]","[2.8901257514953613, 2.8901257514953613]","[0.06833142042160034, 0.4276972711086273]","[411, 4122]"
3,"[12.579470703125, 12.579470703125]","[1884.987060546875, 2220.384521484375]","[-1.1025604009628296, -1.1025604009628296]","[0.9249186515808105, 0.9249186515808105]","[-0.5603189468383789, -0.41332852840423584]","[411, 4122]"
4,[36.1288359375],[2235.69873046875],[0.6598162055015564],[0.3584118187427521],[0.19036155939102173],[4122]
5,"[15.020291015625, 15.020291015625]","[1877.15478515625, 2239.309326171875]","[-0.03431634232401848, -0.03431634232401848]","[1.0283979177474976, 1.0283979177474976]","[-0.2011619210243225, 0.06732072681188583]","[411, 4122]"
6,[14.8352314453125],[1867.1314697265625],[-0.9280846118927002],[2.8990697860717773],[0.6845715045928955],[411]
7,"[12.4003134765625, 12.4003134765625]","[1835.0294189453125, 2314.31494140625]","[-0.4558798670768738, -0.4558798670768738]","[4.075687408447266, 4.075687408447266]","[0.8342868089675903, 0.9141804575920105]","[411, 4122]"
8,[8.800140625],[2174.80078125],[-1.4868755340576172],[0.24635069072246552],[-0.7372457981109619],[4122]
9,[30.092365234375],[2306.049072265625],[0.4342458248138428],[7.426616668701172],[-0.9373416304588318],[4122]


In [15]:
DMesons_df2 = show(0, TTree, ["DMesons_daughterInfo__pt", "DMesons_daughterInfo__eta", "DMesons_daughterInfo__phi", "DMesons_daughterInfo__pdgId"], 75)
DMesons_df2 = remove_empty(DMesons_df2)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [16]:
DPlus = [-321, 211, 211]
LambdaC = [2212, -321, 211]

remove_lambda = []

count = 0
for index, row in DMesons_df2.iterrows():
    if count == 50:
        break
    count+=1
    
    pts = row['DMesons_daughterInfo__pt']
    etas = row['DMesons_daughterInfo__eta']
    phis = row['DMesons_daughterInfo__phi']
    pdgids = row["DMesons_daughterInfo__pdgId"]
    
    #loop through until the first lambda encountered
    for i in range(len(pdgids) - 1):
        #if there are multiple lambda baryons
        if len(pdgids) > 2:
            #stop at the first lambda and keep the higher proton pt valued lambda
            if list(pdgids[i]) == LambdaC:
                l1_pt, l2_pt = pts[i][0], pts[i + 1][0]
                if (l1_pt > l2_pt):
                    #Lambda 1's proton has higher pt
                    remove_lambda.append((index, i + 1))
                elif (l1_pt < l2_pt):
                    #Lambda 2's proton has higher pt!
                    remove_lambda.append((index, i))
                else:
                    raise Exception("Error: Are the proton pts equal?")
remove_lambda

[(32, 2), (46, 1), (48, 1)]

In [17]:
#takes tuples of shape (2,) / (index, column)
def remove_particles_from_df(df, tuple_list):
    for tup in tuple_list:
        index, column = tup
        print(index, column)
        
        new_df = df.copy()
        for i in range(len(DMesons_df.iloc[index, 0][column]) + 1):
            #new_df["DMesons_daughterInfo__pt"] = new_df["DMesons_daughterInfo__pt"].apply(lambda x: print(x))
            val = new_df.iloc[index, i][column]
            print(val)
    df = df.drop(index, axis = 0)
    return df
#df = remove_particles_from_df(DMesons_df, remove_lambda)

#for now just remove the rows
indices = [i for i, x in enumerate(DMesons_df2[DMesons_df2.columns[0]]) if len(x) > 2]
DMesons_df2 = DMesons_df2.drop(indices)
DMesons_df2 = DMesons_df2.reset_index(drop=True)
DMesons_df2["DMesons_daughterInfo__pt"] = DMesons_df2["DMesons_daughterInfo__pt"].apply(divide_by_1000)
DMesons_df2

pd.set_option('display.max_rows', 10)
# pd.set_option('display.max_columns', None)

# print("-----------------")
# print(DMesons_df.iloc[32, 0][2])
# print(DMesons_df.iloc[32, 1][2])
# print(DMesons_df.iloc[32, 2][2])
# print(DMesons_df.iloc[32, 3][2])

In [18]:
#df = pd.concat([DMesons_df, DMesons_df2], axis = 1)
#target = DMesons_df.pop("DMesons_pdgId")

for col in DMesons_df.columns:
    DMesons_df[col] = DMesons_df[col].apply(list)
DMesons_df

,DMesons_pt,DMesons_m,DMesons_eta,DMesons_fitOutput__Lxy,DMesons_costhetastar,DMesons_pdgId
0,[10.047125],[1862.4176025390625],[0.08182407170534134],[2.3395302295684814],[0.25487038493156433],[411]
1,"[18.9937421875, 18.9937421875]","[1853.90625, 2398.886962890625]","[-1.6619627475738525, -1.6619627475738525]","[1.9184143543243408, 1.9184143543243408]","[-0.36730244755744934, -0.0650731548666954]","[411, 4122]"
2,"[10.118494140625, 10.118494140625]","[1874.203125, 2468.1806640625]","[0.041659336537122726, 0.041659336537122726]","[2.8901257514953613, 2.8901257514953613]","[0.06833142042160034, 0.4276972711086273]","[411, 4122]"
3,"[12.579470703125, 12.579470703125]","[1884.987060546875, 2220.384521484375]","[-1.1025604009628296, -1.1025604009628296]","[0.9249186515808105, 0.9249186515808105]","[-0.5603189468383789, -0.41332852840423584]","[411, 4122]"
4,[36.1288359375],[2235.69873046875],[0.6598162055015564],[0.3584118187427521],[0.19036155939102173],[4122]
...,...,...,...,...,...,...
41,"[11.8294248046875, 11.8294248046875]","[1843.660888671875, 2297.289306640625]","[-2.1346840858459473, -2.1346840858459473]","[8.951966285705566, 8.951966285705566]","[0.5174757242202759, 0.7488387823104858]","[411, 4122]"
42,"[22.718240234375, 22.718240234375]","[1832.157958984375, 2250.080810546875]","[0.8391884565353394, 0.8391884565353394]","[3.3735082149505615, 3.3735082149505615]","[-0.33979323506355286, -0.0883156880736351]","[411, 4122]"
43,"[12.4448828125, 12.4448828125]","[1915.7474365234375, 2336.346435546875]","[-1.1050832271575928, -1.1050832271575928]","[0.9720778465270996, 0.9720778465270996]","[0.6463268399238586, 0.8104363679885864]","[411, 4122]"
44,[20.794609375],[2308.2685546875],[-0.20504680275917053],[0.21821393072605133],[-0.09182504564523697],[4122]


In [19]:
# create a new column 'particle_type' based on the pdgId column
func = lambda x: 'only_411' if x == [411] else ('only_4122' if x == [4122] else 'both')
DMesons_df['particle_type'] = DMesons_df['DMesons_pdgId'].apply(func)

# separate the rows based on the new column
only_411 = DMesons_df[DMesons_df['particle_type'] == 'only_411']
only_4122 = DMesons_df[DMesons_df['particle_type'] == 'only_4122']
both = DMesons_df[DMesons_df['particle_type'] == 'both']

only_4122

,DMesons_pt,DMesons_m,DMesons_eta,DMesons_fitOutput__Lxy,DMesons_costhetastar,DMesons_pdgId,particle_type
4,[36.1288359375],[2235.69873046875],[0.6598162055015564],[0.3584118187427521],[0.19036155939102173],[4122],only_4122
8,[8.800140625],[2174.80078125],[-1.4868755340576172],[0.24635069072246552],[-0.7372457981109619],[4122],only_4122
9,[30.092365234375],[2306.049072265625],[0.4342458248138428],[7.426616668701172],[-0.9373416304588318],[4122],only_4122
10,[8.1931796875],[2370.864013671875],[-0.14869287610054016],[0.23419557511806488],[0.6557851433753967],[4122],only_4122
12,[30.220482421875],[2214.49072265625],[-1.783481240272522],[0.1535062938928604],[0.44907552003860474],[4122],only_4122
30,[11.1051123046875],[2233.403564453125],[1.5813356637954712],[0.4592438340187073],[0.7630120515823364],[4122],only_4122
44,[20.794609375],[2308.2685546875],[-0.20504680275917053],[0.21821393072605133],[-0.09182504564523697],[4122],only_4122


In [20]:
# model = Sequential()
# model.add(tf.keras.Input(shape = (9)))
# model.add(Dense(6, activation = "relu"))
# model.add(Dense(1, activation = "softmax"))

# #Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# #Train the model
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# #Evaluate on the testing set
# loss, accuracy = model.evaluate(X_test, y_test)
# print('Test accuracy:', accuracy)

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

only_411 = DMesons_df[DMesons_df['DMesons_pdgId'].apply(lambda x: x == [411])]
only_4122 = DMesons_df[DMesons_df['DMesons_pdgId'].apply(lambda x: x == [4122])]
both = DMesons_df[DMesons_df['DMesons_pdgId'].apply(lambda x: len(x) == 2)]

#Split data
train_411, test_411 = train_test_split(only_411, test_size=0.2)
train_4122, test_4122 = train_test_split(only_4122, test_size=0.2)
train_both, test_both = train_test_split(both, test_size=0.2)

#Combine the training set and testing set for each particle type
train = pd.concat([train_411, train_4122, train_both])
test = pd.concat([test_411, test_4122, test_both])

#Preprocess the input features
scaler = StandardScaler()
#X_train = scaler.fit_transform(train.iloc[:, :-2].values)
#y_train = train['particle_type'].apply(lambda x: 0 if x == 'only_411' else (1 if x == 'only_4122' else 2)).values
#X_test = scaler.transform(test.iloc[:, :-2].values)
#y_test = test['particle_type'].apply(lambda x: 0 if x == 'only_411' else (1 if x == 'only_4122' else 2)).values

In [26]:
#model
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(9,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [ ]:
#Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

#Evaluate testing set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")